### Monday, December 4, 2023

Trying it again ...

pip install exllamav2

docker container start hfpt_Oct28

### Sunday, December 3, 2023

Downloaded the "TheBloke/Python-Code-33B-GPTQ" model yesterday ... ran a quick test this morning. It works!

### Friday, November 17, 2023

https://huggingface.co/TheBloke/Python-Code-33B-GPTQ

In [1]:
# You need this to just target the 4090.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
model_name_or_path = "TheBloke/Python-Code-33B-GPTQ"

In [4]:
!ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  version.txt
models--TheBloke--Python-Code-33B-GPTQ


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [6]:
%%time
# To use a different branch, change revision
# For example: revision="gptq-4bit-128g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             # use_exllama_v2=True, ... this does not work
                                             revision="main")


# Load time
# CPU times: user 19.9 s, sys: 9.32 s, total: 29.3 s
# Wall time: 12.4 s
# 18424MiB VRAM

# Download time
# CPU times: user 1min 32s, sys: 1min 39s, total: 3min 12s
# Wall time: 4h 37s

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


CPU times: user 20.4 s, sys: 8.88 s, total: 29.3 s
Wall time: 11.9 s


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--Python-Code-33B-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 16.9GB to /home/rob/Data3/huggingface/transformers

In [7]:
prompt = "Tell me about AI"
prompt_template=f'''This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''

# print("\n\n*** Generate:")

In [8]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

In [9]:
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print(tokenizer.decode(output[0]))

# 19524 MiB VRAM

# 18.0s
# 21.1s
# 9.5s
# 14.9s

In [14]:
len(output[0])

420

In [11]:
print(tokenizer.decode(output[0]))

<s> This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:
Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a system that can perceive its environment, reason about it, and take actions to achieve its goals. 

In the field of AI, we study how to build such intelligent agents using computers. We use various techniques and algorithms to enable computers to perceive, reason, and act. 

Some popular topics in AI include:

1. Machine Learning: This is the study of algorithms that allow computers to learn from data. We use machine learning algorithms to train computers to identify patterns, classify data, and make predictions.

2. Deep Learning: This is a subfield of machine learning that uses deep neural networks to learn from data. Deep neural networks are inspired by the struc

In [28]:
%%time
minTokens = 2048
maxTokens = minTokens + 128
output2048 = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=maxTokens)
output = output2048[0]
print(len(output))
print(tokenizer.decode(output))


# 38.4s
# 20.8s

538
<s> This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:

AI or Artificial Intelligence is a subfield of Computer Science that studies intelligent agents. An intelligent agent is a software program that can solve problems by using a combination of logic, data, and algorithms. AI researchers study how to design and build such agents, and how to make them more intelligent.

AI has many subfields, including machine learning, deep learning, reinforcement learning, and natural language processing. Each of these subfields focuses on a different aspect of AI, such as how to learn from data, how to reason about complex problems, or how to communicate with humans.

AI is a rapidly growing field, with many new developments and applications. Some of the most exciting areas of AI research include:

1. Self-driving cars: AI is used to control the 

In [10]:
# Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [11]:
print(pipe(prompt_template)[0]['generated_text'])

# 20324 MiB VRAM
# 19906 MiB VRAM

# 15.0s
# 12.9s 
# 15.2s

This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:
Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a program that can perceive its environment and take actions to achieve a goal. 

There are many different types of AI algorithms, each designed to solve a specific problem. Some common types of AI include:

1. Search Algorithms: These algorithms are used to search for a target value in a large dataset. They may be used in web searches, code searches, or database searches.

2. Optimization Algorithms: These algorithms are used to find the optimal solution to a problem. They may be used in scheduling, routing, or resource allocation.

3. Learning Algorithms: These algorithms are used to learn a function from data. They may be used in spam filtering, recommendation systems,